In [1]:
from google.cloud import bigquery
import requests
from google.cloud import storage
import os
import pandas as pd

In [6]:
# URL base de la API
base_url = "https://data.cityofnewyork.us/resource/c3uy-2p5r.json"

# Parámetros para el límite y compensación
limit = 1000  # Número máximo de filas por solicitud
offset = 0    # Compensación inicial
all_data = [] # Lista para almacenar todos los DataFrames

# Descargar datos paginados
while True:
    # Crear la URL con los parámetros de paginación
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    
    # Realizar la solicitud GET a la API
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error en la solicitud: {response.status_code}")
        break
    
    # Leer los datos en un DataFrame de pandas
    data = pd.DataFrame(response.json())
    
    # Si no hay más datos, salir del bucle
    if data.empty:
        break
    
    # Agregar el DataFrame a la lista
    all_data.append(data)
    
    # Incrementar el offset
    offset += limit

# Concatenar todos los DataFrames en uno solo
final_data = pd.concat(all_data, ignore_index=True)

# Guardar el DataFrame en un archivo CSV temporal
local_csv_path = "./ruta/calidad_del_aire.csv"
final_data.to_csv(local_csv_path, index=False)

# Subir el archivo CSV a Google Cloud Storage
bucket_name = "datavision-crudos"
destination_blob_name = "calidad_del_aire.csv"

def upload_to_gcs(csv_path, bucket_name, destination_blob_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(csv_path)
    print(f"Datos subidos a {bucket_name}/{destination_blob_name} en Google Cloud Storage.")

# Ejecuta la subida del archivo a Google Cloud Storage
upload_to_gcs(local_csv_path, bucket_name, destination_blob_name)

# Eliminar el archivo temporal si no es necesario
os.remove(local_csv_path)
print("Archivo temporal eliminado.")


Datos subidos a datavision-crudos/calidad_del_aire.csv en Google Cloud Storage.
Archivo temporal eliminado.
